# Portfolio Rebalancer in Python

Timely and consistent rebalancing is a cornerstone of modern portfolio theory. Rebalancing can magnify returns and reduce long-term risk. This project was inspired by a lack of free tools for simply rebalancing an existing portfolio. Sure, we can all use a worksheet to do the math each time, but why not automate it? With inspiration from the work of kdboller we use Pandas, the Tiingo API, and some simple math to calculate how to optimally rebalance a portfolio given a target allocation.

Steps:
1.Define our current Portfolio (accounttype, time, ticker, shares, cost basis, price)
2.Define our target allocation (ticker, allocation)
3.Factor in any new money being invested
4.Calculate initial transactions needed to hit target allocation
5.Set triggers to rebalance (time or threshold or both)
6.Iteratively determine sells and buys required to get as close as possible to target allocation

References:
https://nbviewer.jupyter.org/github/kdboller/pythonsp500/blob/a7066d998ff046c3cc8b26ece3b0efdf00959d57/Investment%20Portfolio%20Python%20Notebook_03_2018_blog%20example.ipynb

In [ ]:


#Phases:
#1. (done) Rebalance an existing portfolio
#2. (done) Rebalance new money in
#3. Factor in account types and tax consideratons
#4. Track portfolio changes using API (will depend on broker capability)
#5. Email alerts

#Feature Ideas:
#Calculate capital gains and losses (and their tax status LT vs. ST)
#Define allocation at asset-class / domestic-international (or other) levels
#Enable buying/selling fractional shares for asset types that allow it (ETFs, individual stocks, etc..)
#Factor in trading fees
#Convert the top-level allocation to the ticker-level allocation (see: https://docs.google.com/spreadsheets/d/1Ig0sYTdfSqcM2OHJDa7m_vRvQeAQO823s2Nw2rGmL5g/edit#gid=628577776)
#Calculate Tax Location for assets (define optimal tax location depending on the security)
#Calculate Tax Loss Harvesting (maybe a separate app)
#Decide on optimal rebalance strategy (sell assets, invest additional assets, factor in taxable accounts vs. tax deferred)

In [100]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import datetime
import decimal
from pandas_datareader import data as pdr
#import fix_yahoo_finance as yf
#yf.pdr_override() # <== Fix for yahoo data
from keys import tiingo_key
now = datetime.datetime.now()

In [147]:
#Settings
new_money_in = 10000
#Set our rebalance threshold
rebal_threshold = .05 #allowable allocation drift
rebal_timeframe = 180 #in days

In [151]:
#Define target and current allocations
#create our target allocation
columns_t = ['ticker','allocation_target']
positions_t = [['VTSAX',0.5652],
             ['VIGAX',0.0131],
             ['VSMAX',0.0066],
             ['VSEQX',0.0066],
             ['VWIGX',0.0507],
             ['VTRIX',0.0507],
             ['VTIAX',0.1521],
             ['VBTLX',0.035],
             ['VTABX',0.015],
             ['VGSLX',0.05],
             ['VNQI',0.01],
             ['VDE',0.03],
             ['GLD',0.015]]

#set our current portfolio
columns_c = ['accounttype','lastrebaldate','ticker','basisdate','costbasis','shares']
positions_c = [['TAXB','2018-11-16','VTSAX','2012-10-27',120.20,2],
             ['401K','2018-11-16','VIGAX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VSMAX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VSEQX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VWIGX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VTRIX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VTIAX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VBTLX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VTABX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VGSLX','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VNQI','2017-1-02',56.70,14.5],
             ['401K','2018-11-16','VDE','2017-1-02',56.70,14.5],
            ['401K','2018-11-16','AAPL','2017-1-02',56.70,14.5]]

In [153]:
#lookup table for account type abbreviations
accounttypes = {'TAXB':'Taxable Brokerage', '401K':'401k', 'RIRA':'Roth-IRA', 'TIRA':'Traditional-IRA'}

#initialize target portfolio
targetalloc = pd.DataFrame(columns = columns_t, data = positions_t)
total=decimal.Decimal(targetalloc.allocation_target.sum())
if round(total,4) != 1:
    print('Target Allocation not 100% : {}'.format(int(total)))

#initialize current portfolio
start_port = pd.DataFrame(columns = columns_c, data = positions_c)
start_port.lastrebaldate = pd.to_datetime(start_port.lastrebaldate)
start_port.basisdate = pd.to_datetime(start_port.basisdate)

#Define list of distinct tickers we care about
tickers = set(targetalloc.ticker.unique().tolist()+start_port.ticker.unique().tolist())

In [4]:
#Next we pull the latest prices from Tiingo (YahooFinance is buggy, and IEX does not contain mutual fund data)
#Tiingo limits for free API: 500 unique tickers ever, 500 requests/hr, 20,000 requests/day
#https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#tiingo
#Tiingo API key required: set 'tiingo_key' value in python file called 'keys.py' in same directory as this script
now = datetime.datetime.now()
yesterday = now - datetime.timedelta(3) #avoids weekends with no data - need better weekend detection
start = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)
end = datetime.datetime(now.year, now.month, now.day)

bad_tickers = []
for i, t in enumerate(tickers):
    try:
        if i==0:
            ohlc = pdr.get_data_tiingo(t, api_key=tiingo_key).tail(1).close
        else:
            ohlc = ohlc.append(pdr.get_data_tiingo(t, api_key=tiingo_key).tail(1).close)
    except:
        bad_tickers.append(t)
        
#print(bad_tickers)
ohlc = ohlc.to_frame(name='close')

#drop our date index since its only the latest data
ohlc2=ohlc.reset_index(level=1, drop=True)

In [155]:
#concatenate target allocation and latest prices with our portfolio
start_port_c = pd.merge(start_port, targetalloc, on ='ticker', how ='outer')
final_port = pd.merge(start_port_c, ohlc2, left_on ='ticker', right_index = True, how = 'left')

#set target to zero for tickers held but not present in our target allocation, set initial basisdate and costbasis for new securities entering the portfolio
final_port.fillna(value = {'allocation_target':0.0,'shares':0.0,'basisdate':pd.to_datetime(now.strftime("%Y-%m-%d")),'costbasis':final_port.close},inplace = True)

#calculate holding values and current allocation
final_port['value'] = final_port.close * final_port.shares #calculate value as price x shares
final_port.loc[final_port.value.isna() & final_port.shares.isna(),['value']]=0.0 #for securities not currently held but in our target (and close price failed to return), establish zero value
final_port['allocation'] = final_port.value / final_port.value.sum()
final_port['correction'] = final_port.allocation_target - final_port.allocation

In [156]:
#factor in any new money entering the portfolio and determine necessary changes in value and shares
#this method rebalances and factors in new money simultaneously - there may be more efficient methods
total_val = final_port.value.sum()
final_port['new_money_in'] = new_money_in * final_port.allocation_target #Account for new money in
final_port['value_chg'] = (total_val * final_port.correction) + final_port.new_money_in
final_port['shares_chg'] = final_port.value_chg / final_port.close
final_port.loc[final_port.value_chg.isna() & final_port.shares > 0,['shares_chg']]=-final_port.shares #sell all shares of securities not in our target portfolio

#round off shares to whole numbers, except when we are fully exiting a position
final_port['shares_chg_round'] = final_port.shares_chg
final_port = final_port.astype({'shares_chg_round': int})
final_port['final_shares_chg'] = final_port.shares_chg
final_port.loc[final_port.shares_chg+final_port.shares!=0,['final_shares_chg']]=final_port.shares_chg_round*1.0
final_port.drop(['shares_chg_round'],axis=1,inplace=True)

#create timedelta int column
final_port['timedelta'] = (final_port.lastrebaldate - pd.to_datetime(now.strftime("%Y-%m-%d"))).dt.days
final_port.timedelta.fillna(0,inplace=True)

#define rebalance flags to determine if we must rebalance
final_port['rebal_flag_thresh'] = np.where((abs(final_port.correction)<=rebal_threshold) & (final_port.allocation > 0),0,1)
final_port['rebal_flag_time'] = np.where(final_port.timedelta >= rebal_timeframe,1,0)
final_port['rebal_flag'] = np.where(final_port.rebal_flag_thresh + final_port.rebal_flag_time >= 1,1,0)

#These rows do not meet our rebalance threshold or timeframe cutoff - so we should set their change values to 0
#The exception is if new_money_in must be considered, in which case we will still rebalance by investing the new money
final_port.loc[(final_port.rebal_flag==0) & (final_port.new_money_in==0), ['value_chg','shares_chg','final_shares_chg']] = 0

#Calculate initial new shares and values
final_port['new_shares'] = (final_port.shares + final_port.final_shares_chg)
final_port['new_value'] = final_port.new_shares * final_port.close #due to share rounding, there will be slight variance vs. portfolio starting value
final_port['new_value_chg'] = final_port.final_shares_chg * final_port.close

In [157]:
final_port

,accounttype,lastrebaldate,ticker,basisdate,costbasis,shares,allocation_target,close,value,allocation,correction,new_money_in,value_chg,shares_chg,final_shares_chg,timedelta,rebal_flag_thresh,rebal_flag_time,rebal_flag,new_shares,new_value,new_value_chg
0,TAXB,2018-11-16,VTSAX,2012-10-27,120.20,2.0,0.5652,65.69,131.380,0.012442,0.552758,5652.0,11488.597886,174.891123,174.0,-23.0,1,0,1,176.0,11561.440,11430.060
1,401K,2018-11-16,VIGAX,2017-01-02,56.70,14.5,0.0131,72.55,1051.975,0.099628,-0.086528,131.0,-782.651379,-10.787752,-10.0,-23.0,1,0,1,4.5,326.475,-725.500
2,401K,2018-11-16,VSMAX,2017-01-02,56.70,14.5,0.0066,68.00,986.000,0.093380,-0.086780,66.0,-850.310237,-12.504562,-12.0,-23.0,1,0,1,2.5,170.000,-816.000
3,401K,2018-11-16,VSEQX,2017-01-02,56.70,14.5,0.0066,31.71,459.795,0.043545,-0.036945,66.0,-324.105237,-10.220916,-10.0,-23.0,0,0,0,4.5,142.695,-317.100
4,401K,2018-11-16,VWIGX,2017-01-02,56.70,14.5,0.0507,27.22,394.690,0.037379,0.013321,507.0,647.654089,23.793317,23.0,-23.0,0,0,0,37.5,1020.750,626.060
5,401K,2018-11-16,VTRIX,2017-01-02,56.70,14.5,0.0507,34.87,505.615,0.047884,0.002816,507.0,536.729089,15.392288,15.0,-23.0,0,0,0,29.5,1028.665,523.050
6,401K,2018-11-16,VTIAX,2017-01-02,56.70,14.5,0.1521,26.13,378.885,0.035882,0.116218,1521.0,2748.147266,105.172111,105.0,-23.0,1,0,1,119.5,3122.535,2743.650
7,401K,2018-11-16,VBTLX,2017-01-02,56.70,14.5,0.0350,10.36,150.220,0.014227,0.020773,350.0,569.346925,54.956267,54.0,-23.0,0,0,0,68.5,709.660,559.440
8,401K,2018-11-16,VTABX,2017-01-02,56.70,14.5,0.0150,21.97,318.565,0.030170,-0.015170,150.0,-10.179175,-0.463322,0.0,-23.0,0,0,0,14.5,318.565,0.000
9,401K,2018-11-16,VGSLX,2017-01-02,56.70,14.5,0.0500,116.36,1687.220,0.159789,-0.109789,500.0,-659.267250,-5.665755,-5.0,-23.0,1,0,1,9.5,1105.420,-581.800


In [158]:
#create a balance counter initially set to the value of new money in (since this is available to invest)
bal = new_money_in

#next we use a value-sorted portfolio to iteratively sell/buy to get our final execution strategy
for index, row in final_port.sort_values(by='new_value_chg',ascending=True).iterrows():
    bal-=row.new_value_chg
    if bal > 0:
        final_port.loc[index,'execution_value'] = row.new_value
        final_port.loc[index,'execution_shares_chg'] = row.final_shares_chg
    else:
        final_port.loc[index,'execution_value'] = row.new_value + bal
        final_port.loc[index,'execution_shares_chg'] = int((row.new_value + bal)/row.close)
    print('Evaluating {} {}, new bal is {}'.format(row.ticker,row.new_value_chg,bal))

#calculate final values for execution
final_port['execution_value_chg'] = final_port.execution_shares_chg * final_port.close
final_port['execution_shares'] = final_port.execution_shares_chg + final_port.shares
final_port['execution_allocation'] = final_port.execution_value / final_port.execution_value.sum()

#double check our work - the final portfolio should be well within the rebalance threshold we defined
if len(final_port[(final_port.execution_allocation - final_port.allocation_target) > rebal_threshold]) > 0:
    print('ERROR: Target Portfolio not achieved')
else:
    print('Allocation Success!')

Evaluating AAPL -2443.105, new bal is 12443.105
Evaluating VSMAX -816.0, new bal is 13259.105
Evaluating VIGAX -725.5, new bal is 13984.605
Evaluating VDE -608.3000000000001, new bal is 14592.904999999999
Evaluating VGSLX -581.8, new bal is 15174.704999999998
Evaluating VNQI -545.9000000000001, new bal is 15720.604999999998
Evaluating VSEQX -317.1, new bal is 16037.704999999998
Evaluating VTABX 0.0, new bal is 16037.704999999998
Evaluating GLD 236.18, new bal is 15801.524999999998
Evaluating VTRIX 523.05, new bal is 15278.474999999999
Evaluating VBTLX 559.4399999999999, new bal is 14719.034999999998
Evaluating VWIGX 626.06, new bal is 14092.974999999999
Evaluating VTIAX 2743.65, new bal is 11349.324999999999
Evaluating VTSAX 11430.06, new bal is -80.73500000000058
Allocation Success!


In [159]:
final_port

,accounttype,lastrebaldate,ticker,basisdate,costbasis,shares,allocation_target,close,value,allocation,correction,new_money_in,value_chg,shares_chg,final_shares_chg,timedelta,rebal_flag_thresh,rebal_flag_time,rebal_flag,new_shares,new_value,new_value_chg,execution_value,execution_shares_chg,execution_value_chg,execution_shares,execution_allocation
0,TAXB,2018-11-16,VTSAX,2012-10-27,120.20,2.0,0.5652,65.69,131.380,0.012442,0.552758,5652.0,11488.597886,174.891123,174.0,-23.0,1,0,1,176.0,11561.440,11430.060,11480.705,174.0,11430.060,176.0,0.558426
1,401K,2018-11-16,VIGAX,2017-01-02,56.70,14.5,0.0131,72.55,1051.975,0.099628,-0.086528,131.0,-782.651379,-10.787752,-10.0,-23.0,1,0,1,4.5,326.475,-725.500,326.475,-10.0,-725.500,4.5,0.015880
2,401K,2018-11-16,VSMAX,2017-01-02,56.70,14.5,0.0066,68.00,986.000,0.093380,-0.086780,66.0,-850.310237,-12.504562,-12.0,-23.0,1,0,1,2.5,170.000,-816.000,170.000,-12.0,-816.000,2.5,0.008269
3,401K,2018-11-16,VSEQX,2017-01-02,56.70,14.5,0.0066,31.71,459.795,0.043545,-0.036945,66.0,-324.105237,-10.220916,-10.0,-23.0,0,0,0,4.5,142.695,-317.100,142.695,-10.0,-317.100,4.5,0.006941
4,401K,2018-11-16,VWIGX,2017-01-02,56.70,14.5,0.0507,27.22,394.690,0.037379,0.013321,507.0,647.654089,23.793317,23.0,-23.0,0,0,0,37.5,1020.750,626.060,1020.750,23.0,626.060,37.5,0.049650
5,401K,2018-11-16,VTRIX,2017-01-02,56.70,14.5,0.0507,34.87,505.615,0.047884,0.002816,507.0,536.729089,15.392288,15.0,-23.0,0,0,0,29.5,1028.665,523.050,1028.665,15.0,523.050,29.5,0.050035
6,401K,2018-11-16,VTIAX,2017-01-02,56.70,14.5,0.1521,26.13,378.885,0.035882,0.116218,1521.0,2748.147266,105.172111,105.0,-23.0,1,0,1,119.5,3122.535,2743.650,3122.535,105.0,2743.650,119.5,0.151881
7,401K,2018-11-16,VBTLX,2017-01-02,56.70,14.5,0.0350,10.36,150.220,0.014227,0.020773,350.0,569.346925,54.956267,54.0,-23.0,0,0,0,68.5,709.660,559.440,709.660,54.0,559.440,68.5,0.034518
8,401K,2018-11-16,VTABX,2017-01-02,56.70,14.5,0.0150,21.97,318.565,0.030170,-0.015170,150.0,-10.179175,-0.463322,0.0,-23.0,0,0,0,14.5,318.565,0.000,318.565,0.0,0.000,14.5,0.015495
9,401K,2018-11-16,VGSLX,2017-01-02,56.70,14.5,0.0500,116.36,1687.220,0.159789,-0.109789,500.0,-659.267250,-5.665755,-5.0,-23.0,1,0,1,9.5,1105.420,-581.800,1105.420,-5.0,-581.800,9.5,0.053768


In [161]:
#Assemble our final output
display_port = final_port[['accounttype','lastrebaldate','ticker','costbasis','shares','close','allocation','allocation_target','execution_value_chg','execution_shares_chg','execution_shares','execution_value','execution_allocation']]

#To complete rebalancing investor should transact the shares in the 'execution_shares_chg' column in any order
display_port

,accounttype,lastrebaldate,ticker,costbasis,shares,close,allocation,allocation_target,execution_value_chg,execution_shares_chg,execution_shares,execution_value,execution_allocation
0,TAXB,2018-11-16,VTSAX,120.20,2.0,65.69,0.012442,0.5652,11430.060,174.0,176.0,11480.705,0.558426
1,401K,2018-11-16,VIGAX,56.70,14.5,72.55,0.099628,0.0131,-725.500,-10.0,4.5,326.475,0.015880
2,401K,2018-11-16,VSMAX,56.70,14.5,68.00,0.093380,0.0066,-816.000,-12.0,2.5,170.000,0.008269
3,401K,2018-11-16,VSEQX,56.70,14.5,31.71,0.043545,0.0066,-317.100,-10.0,4.5,142.695,0.006941
4,401K,2018-11-16,VWIGX,56.70,14.5,27.22,0.037379,0.0507,626.060,23.0,37.5,1020.750,0.049650
5,401K,2018-11-16,VTRIX,56.70,14.5,34.87,0.047884,0.0507,523.050,15.0,29.5,1028.665,0.050035
6,401K,2018-11-16,VTIAX,56.70,14.5,26.13,0.035882,0.1521,2743.650,105.0,119.5,3122.535,0.151881
7,401K,2018-11-16,VBTLX,56.70,14.5,10.36,0.014227,0.0350,559.440,54.0,68.5,709.660,0.034518
8,401K,2018-11-16,VTABX,56.70,14.5,21.97,0.030170,0.0150,0.000,0.0,14.5,318.565,0.015495
9,401K,2018-11-16,VGSLX,56.70,14.5,116.36,0.159789,0.0500,-581.800,-5.0,9.5,1105.420,0.053768
